In [1]:
import pandas as pd
import pickle
import numpy as np
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm

train = pd.read_csv('../input/train.csv', parse_dates = ['activation_date'])
test = pd.read_csv('../input/test.csv', parse_dates = ['activation_date'])
train_active = pd.read_csv('../input/train_active.csv',parse_dates = ['activation_date'])
test = pd.read_csv('../input/test.csv')
test_active = pd.read_csv('../input/test_active.csv',parse_dates = ['activation_date'])
train_periods = pd.read_csv('../input/periods_train.csv', parse_dates=['activation_date','date_from', 'date_to'])
test_periods = pd.read_csv('../input/periods_test.csv', parse_dates=['activation_date','date_from', 'date_to'])

df_all = pd.concat([
    train,
    train_active,
    test,
    test_active
]).reset_index(drop=True)
df_all.drop_duplicates(['item_id'], inplace=True)


In [2]:
df_all.head()

,activation_date,category_name,city,deal_probability,description,image,image_top_1,item_id,item_seq_number,param_1,param_2,param_3,parent_category_name,price,region,title,user_id,user_type
0,2017-03-28 00:00:00,Товары для детей и игрушки,Екатеринбург,0.12789,"Кокон для сна малыша,пользовались меньше месяц...",d10c7e016e03247a3bf2d13348fe959fe6f436c1caf64c...,1008.0,b912c3c6a6ad,2.0,Постельные принадлежности,NaN,NaN,Личные вещи,400.0,Свердловская область,Кокоби(кокон для сна),e00f8ff2eaf9,Private
1,2017-03-26 00:00:00,Мебель и интерьер,Самара,0.00000,"Стойка для одежды, под вешалки. С бутика.",79c9392cc51a9c81c6eb91eceb8e552171db39d7142700...,692.0,2dac0150717d,19.0,Другое,NaN,NaN,Для дома и дачи,3000.0,Самарская область,Стойка для Одежды,39aeb48f0017,Private
2,2017-03-20 00:00:00,Аудио и видео,Ростов-на-Дону,0.43177,"В хорошем состоянии, домашний кинотеатр с blu ...",b7f250ee3f39e1fedd77c141f273703f4a9be59db4b48a...,3032.0,ba83aefab5dc,9.0,"Видео, DVD и Blu-ray плееры",NaN,NaN,Бытовая электроника,4000.0,Ростовская область,Philips bluray,91e2f88dd6e3,Private
3,2017-03-25 00:00:00,Товары для детей и игрушки,Набережные Челны,0.80323,Продам кресло от0-25кг,e6ef97e0725637ea84e3d203e82dadb43ed3cc0a1c8413...,796.0,02996f1dd2ea,286.0,Автомобильные кресла,NaN,NaN,Личные вещи,2200.0,Татарстан,Автокресло,bf5cccea572d,Company
4,2017-03-16 00:00:00,Автомобили,Волгоград,0.20797,Все вопросы по телефону.,54a687a3a0fc1d68aed99bdaaf551c5c70b761b16fd0a2...,2264.0,7c90be56d2ab,3.0,С пробегом,ВАЗ (LADA),2110,Транспорт,40000.0,Волгоградская область,"ВАЗ 2110, 2003",ef50846afc0b,Private


In [4]:
df_all["activation_date"] = pd.to_datetime(df_all["activation_date"])

In [5]:
import numpy as np
df_all['wday'] = df_all['activation_date'].dt.weekday

In [6]:

df_all['price'] = np.log1p(df_all['price'])

all_periods = pd.concat([
    train_periods,
    test_periods
])


In [7]:
all_periods['days_up'] = all_periods['date_to'].dt.dayofyear - all_periods['date_from'].dt.dayofyear
all_periods['days_total'] = all_periods['date_to'].dt.dayofyear - all_periods['activation_date'].dt.dayofyear


In [8]:
def agg(df,agg_cols):
    for c in tqdm(agg_cols):
        new_feature = '{}_{}_{}'.format('_'.join(c['groupby']), c['agg'], c['target'])
        gp = df.groupby(c['groupby'])[c['target']].agg(c['agg']).reset_index().rename(index=str, columns={c['target']:new_feature})
        df = df.merge(gp,on=c['groupby'],how='left')
    return df

agg_cols = [
    {'groupby': ['item_id'], 'target':'days_up', 'agg':'count'},
    {'groupby': ['item_id'], 'target':'days_up', 'agg':'sum'},   
    {'groupby': ['item_id'], 'target':'days_total', 'agg':'sum'},  
]

all_periods = agg(all_periods,agg_cols)

all_periods.drop_duplicates(['item_id'], inplace=True)
all_periods.drop(['activation_date','date_from','date_to','days_up','days_total'],axis=1, inplace=True)
all_periods.reset_index(drop=True,inplace=True)

df_all = df_all.merge(all_periods, on='item_id', how='left')

df_all.to_pickle('./data/df_all.pkl')

100%|██████████| 3/3 [06:08<00:00, 122.85s/it]
